# Segmenting and Clustering Neighborhoods in Toronto

Required to explore and cluster the neighborhoods in Toronto.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import requests 
from bs4 import BeautifulSoup 

print('Libraries imported.')

Libraries imported.


Downloading data

In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, features='html')
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')

Page Scrapped.


Extracting all important fields

In [3]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string.rstrip());      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1].rstrip())
                else:
                    neighborhoods.append(cell.string.rstrip()); 
                columnNum = 1
                
print('Data Collected.')

Data Collected.


Creating a dataframe and importing data

In [4]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors_raw = pd.DataFrame(columns=column_names)

for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors_raw = neighbors_raw.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)

neighbors_raw

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
...,...,...,...
205,M8Z,Etobicoke,Kingsway Park South West
206,M8Z,Etobicoke,Mimico NW
207,M8Z,Etobicoke,The Queensway West
208,M8Z,Etobicoke,Royal York South West


Clearing data

In [5]:
# neighbors_raw.groupby('PostalCode')[['Borough','Neighborhood']].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
# neighbors_raw.groupby('PostalCode')[['Borough','Neighborhood']].apply(lambda g: g.values.tolist()).to_dict()

# find unique value of Neighborhood
tmp_Borough=neighbors_raw.groupby('PostalCode')['Borough'].unique().apply(list).to_dict()
# finding and grouping values of Neighborhood for the same PostaCode
tmp_Neighborhood=neighbors_raw.groupby('PostalCode')['Neighborhood'].apply(list).to_dict()

ss=pd.DataFrame.from_dict(list(tmp_Borough.items()))
ss.columns=column_names[0:2]
sss=pd.DataFrame.from_dict(list(tmp_Neighborhood.items()))
sss.columns=[column_names[0],column_names[2]]
ss['Neighborhood']=sss['Neighborhood']


#ss.applymap(lambda x: x[0] if isinstance(x, list) else x)

ss['Borough']=ss['Borough'].str.get(0)
ss["Neighborhood"]=ss["Neighborhood"].apply(lambda x: ", ".join(x) if isinstance(x, list) else x)

neighbors_raw = ss
neighbors_raw

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [6]:
neighbors_raw.shape

(103, 3)